<p align="center">
  <img src="../logo_ufjf.png" width="200"><br>
  <b>UNIVERSIDADE FEDERAL DE JUIZ DE FORA</b><br>
  Programa de Pós-Graduação em Matemática<br>
  Análise Numérica I<br><br>
  Luís Karlos Mendes de Oliveira<br><br>
  <b>Atividade 01:</b> Método de Euler Explícito (MEE) e Método da Série de Taylor (MST)<br><br>
  <p align="left">Prof.: Sandro Rodrigues Mazorche</p>
  <p align="right">16 de setembro de 2025</p>
</p>


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import mylib as m
import os
import ipynbname

diretorio_tabela = f"tables/{ipynbname.name()}"
diretorio_imagem = f"img/{ipynbname.name()}"
os.makedirs(diretorio_tabela, exist_ok=True)
os.makedirs(diretorio_imagem, exist_ok=True)


## **Questão 1 - e)**
$\begin{cases}
  x'(t) = 4t\sqrt{(x)} \\
  x(0) = 1
\end{cases}$

Solução exata: $(t^2 + 1)^2$


### Método de Euler Explícito

In [ ]:
resultados = {}


In [ ]:
f = lambda t, x: 4*t*np.sqrt(x)
condicao_inicial = [0, 1]

T = 1
dt = .025
tempos_desejados = [0, .5, 1]

print(f'Resolvendo o PVI com dt = {dt} ...')
t_euler, y_euler = m.metodo_euler_explicito(f, condicao_inicial, T, dt)
y_exato = (t_euler**2 + 1)**2

dados = pd.DataFrame({
  't': t_euler,
  'x_exato(t)': y_exato,
  'x(t)': y_euler,
  'err_loc': m.erro_local(y_exato ,y_euler),
  'err_rel (%)': m.erro_relativo(y_exato ,y_euler)*100
})

resultados[f'h={dt}'] = dados

for nome, df in resultados.items():
  mascara = df['t'].round(5).isin(tempos_desejados)
  print("\n" + "="*60)
  print(f"Resultados para {nome}")
  print("="*60)
  print(df[mascara])

  codigo_latex = df[mascara].to_latex(
    escape=True,
    index=False,
    caption=f"Resultados da simulação para {nome} no intervalo [0, {T}].",
    label=f"tab:{nome.replace('=', '_')}",
    column_format="ccccc",
    float_format="%.6f"
  )

  codigo_latex = (
    "\\begin{table}[H]\n"
    "\\centering\n"
    + codigo_latex.replace("\\begin{table}", "").replace("\\end{table}", "")
    + "\\end{table}"
  )

  nome_arquivo = f"{diretorio_tabela}/tabela_euler_{nome.replace('=', '_')}_1a.tex"

  with open(nome_arquivo, 'w') as f:
    f.write(codigo_latex)

  print(f"✅ Tabela salva com sucesso no arquivo: {nome_arquivo}")

fig, ax = plt.subplots(figsize=(12, 7))
t_curva_exata = np.linspace(0, T, 200)
y_curva_exata = (t_curva_exata**2 + 1)**2
ax.plot(t_curva_exata, y_curva_exata, 'k-', label='Solução Exata $x(t) = (t^2 + 1)^2$')
for nome, df in resultados.items():
    ax.plot(df['t'], df['x(t)'], '.-', label=f'{nome}')

#ax.set_title('Comparação do Método de Euler com a Solução Exata', fontsize=16)
ax.set_xlabel('t', fontsize=14)
ax.set_ylabel('x(t)', fontsize=14)
ax.legend(fontsize=14)
ax.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.autoscale(enable=True, axis='x', tight=True)
nome_arquivo = f"{diretorio_imagem}/euler_vs_exata.pdf"
plt.savefig(nome_arquivo)
print(f"✅ Gráfico salvo com sucesso no arquivo: {nome_arquivo}")
plt.show()


### Método da Série de Taylor

In [ ]:
resultados_mst = {}


In [ ]:
f0 = lambda t, x: 4 * t**3 + 4 * t
f1 = lambda t, x: 12 * t**2 + 4
f2 = lambda t, x: 24 * t
f3 = lambda t, x: 24.0
f4 = lambda t, x: 0.0

derivadas_calculadas = [f0, f1, f2, f3, f4]
condicao_inicial = [0, 1]
T = 1
dt = .05
ordem = 4
tempos_desejados = [0, .5, 1]

# Calcular a solução exata
t_exato = np.linspace(0, T, 1000)
y_exato = (t_exato**2 + 1)**2

# Calcular a solução com o Método da Série de Taylor
t_out, y_taylor = m.metodo_serie_taylor(derivadas_calculadas[:ordem], condicao_inicial, T, dt)
y_exato_interp = (t_out**2 + 1)**2  # Usar a fórmula exata para os tempos de t_out
e_rel = m.erro_relativo(y_exato_interp, y_taylor) * 100
e_loc = m.erro_local(y_exato_interp, y_taylor)

dados_mst = pd.DataFrame({
    't': t_out,
    'x_exato(t)': y_exato_interp,
    'x(t)': y_taylor,
    'err_loc': e_loc,
    'err_rel (%)': e_rel
})

resultados_mst[f'ordem={ordem}'] = dados_mst

for nome, df in resultados_mst.items():
    mascara = df['t'].round(5).isin(tempos_desejados)
    print("\n" + "="*60)
    print(f"Resultados do MST para {nome}")
    print("="*60)
    print(df[mascara])

    codigo_latex = df[mascara].to_latex(
        escape=True,
        index=False,
        caption=f"Resultados da simulação para {nome} no intervalo [0, {T}].",
        label=f"tab:{nome.replace('=', '_')}",
        column_format="ccccc",
        float_format="%.6f"
    )

    codigo_latex = (
        "\\begin{table}[H]\n"
        "\\centering\n"
        + codigo_latex.replace("\\begin{table}", "").replace("\\end{table}", "")
        + "\\end{table}"
    )

    nome_arquivo = f"{diretorio_tabela}/tabela_mst_{nome.replace('=', '_')}.tex"

    with open(nome_arquivo, 'w') as f:
        f.write(codigo_latex)

    print(f"✅ Tabela salva com sucesso no arquivo: {nome_arquivo}")

fig, ax = plt.subplots(figsize=(12, 7))
t_curva_exata = np.linspace(0, T, 200)
y_curva_exata = (t_curva_exata**2 + 1)**2
ax.plot(t_curva_exata, y_curva_exata, 'k-', linewidth=2, label='Solução Exata $x(t) = (t^2 + 1)^2$')
for nome, df in resultados_mst.items():
    ax.plot(df['t'], df['x(t)'], '.-', label=f'{nome}')

ax.set_xlabel('t', fontsize=14)
ax.set_ylabel('x(t)', fontsize=14)
ax.legend(fontsize=14)
ax.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.autoscale(enable=True, axis='x', tight=True)
nome_arquivo = f"{diretorio_imagem}/taylor_vs_exata.pdf"
plt.savefig(nome_arquivo)
print(f"✅ Gráfico salvo com sucesso no arquivo: {nome_arquivo}")
plt.show()
